In [1]:
import opendatasets as od
import pandas
 
od.download("https://www.kaggle.com/competitions/feather-in-focus/data")

Skipping, found downloaded files in "./feather-in-focus" (use force=True to force download)


In [1]:
#https://www.kaggle.com/code/chekoduadarsh/starters-guide-convolutional-xgboost/notebook
#https://www.kaggle.com/code/pedrolucasbritodes/bird-image-classification-cnn-89-accuracy/notebook
#https://stats.stackexchange.com/questions/404809/is-it-advisable-to-use-output-from-a-ml-model-as-a-feature-in-another-ml-model
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
import tensorflow as tf

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

#how many epchos still revert
patience1 = 1000

# these are the decaying laying edit thing. think of a cool name 
patience2 = 4
always_keep_percent_locked = 20
start_unlocking_at = 80
decay_by = 10

blur_number = 4
base_learning_rate = 0.001 
#number before fine_tuning
number_epcho1 = 20
#number after fine_tuning
number_epcho2 = 20
number_class = 200
#needs to be the same number of classes
output_Neurons = 200
Neurons = 512
drop_out_rate = 0.3
image_shape = (224,224)
image_shape_full = (224,224,3)
current_directory = "feather-in-focus"

2023-12-07 16:34:37.803276: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 16:34:37.984116: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#loading files 
class_names2 = np.load('feather-in-focus/class_names.npy', allow_pickle=True).item()

#load all files because why not
test_image_path = pd.read_csv('feather-in-focus/test_images_path.csv')
test_image_path['total_path'] = current_directory + "/test_images" + test_image_path['image_path'] 
test_image_path['total_path'] = test_image_path['total_path'].str.replace("\\", "/" )
test_image = pd.read_csv('feather-in-focus/test_images_sample.csv')

#importing training data 
train_image = pd.read_csv('feather-in-focus/train_images.csv')
train_image['total_path'] = current_directory + "/train_images" + train_image['image_path']
train_image['total_path'] = train_image['total_path'].str.replace("\\", "/" )
train_image['label'] = train_image['label'].apply(str)
train_image['short_name'] = train_image['image_path'].str.rsplit('/', n=1).str[-1]
train_image.head()



/scratch-local/scur2079.4603773/ipykernel_1066137/4078724003.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test_image_path['total_path'] = test_image_path['total_path'].str.replace("\\", "/" )
/scratch-local/scur2079.4603773/ipykernel_1066137/4078724003.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train_image['total_path'] = train_image['total_path'].str.replace("\\", "/" )


,image_path,label,total_path,short_name
0,/train_images/1.jpg,1,feather-in-focus/train_images/train_images/1.jpg,1.jpg
1,/train_images/2.jpg,1,feather-in-focus/train_images/train_images/2.jpg,2.jpg
2,/train_images/3.jpg,1,feather-in-focus/train_images/train_images/3.jpg,3.jpg
3,/train_images/4.jpg,1,feather-in-focus/train_images/train_images/4.jpg,4.jpg
4,/train_images/5.jpg,1,feather-in-focus/train_images/train_images/5.jpg,5.jpg


In [3]:
# we need to add a k-folds, i have stratified so it works a bit better
train, valid = train_test_split(train_image, 
                    test_size=0.2,
                    stratify = train_image['label'],
                    random_state=420)


In [4]:
#deleting images if they are there 

folder_path = "custom_aug"  # Replace 'path_to_folder' with the actual path to your folder

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    # Check if the file is an image (you might want to refine this check based on your specific file types)
    if os.path.isfile(file_path) and (filename.endswith('.jpg') or filename.endswith('.png')):
        os.remove(file_path)
        #print(f"Deleted: {file_path}")

In [5]:
#custom Augmentation 
import cv2
from PIL import Image, ImageFilter

persp_train_dataframe = pd.DataFrame(columns=['image_path','label','total_path','short_name'])
persp_valid_dataframe = pd.DataFrame(columns=['image_path','label','total_path','short_name'])

gs_train_dataframe = pd.DataFrame(columns=['image_path','label','total_path','short_name'])
gs_valid_dataframe = pd.DataFrame(columns=['image_path','label','total_path','short_name'])

blur_train_dataframe = pd.DataFrame(columns=['image_path','label','total_path','short_name'])
blur_valid_dataframe = pd.DataFrame(columns=['image_path','label','total_path','short_name'])

def custom_blur(dataframe, label, name, location, folder,final_df, blur_number):
    img = Image.open(dataframe[location]).filter(ImageFilter.GaussianBlur(blur_number))
    img.save(r'{}/blur_{}'.format(folder,dataframe[name]))

    return pd.DataFrame({"image_path": ['{}/blur_{}'.format(folder,dataframe[name])],
                      'label': [dataframe[label]],
                      'total_path': ['{}/blur_{}'.format(folder,dataframe[name])],
                      'short_name': ['blur_{}'.format(dataframe[name])]
                      })

def custom_grey_scale(dataframe, label, name, location, folder,final_df):
    img = Image.open(dataframe[location]).convert('L')
    img.save(r'{}/gs_{}'.format(folder,dataframe[name]))

    return pd.DataFrame({"image_path": ['{}/gs_{}'.format(folder,dataframe[name])],
                      'label': [dataframe[label]],
                      'total_path': ['{}/gs_{}'.format(folder,dataframe[name])],
                      'short_name': ['gs_{}'.format(dataframe[name])]
                      })

def custom_getPerspectiveTransform(dataframe, label, name, location, folder,final_df):
    
    
    input_image = cv2.imread(dataframe[location])  # Replace 'path_to_your_image.jpg' with the actual path to your image
    # need to variability
    height, width = input_image.shape[:2]
      
   # Define destination points based on image size (20% of width and 30% of height)
    objPoints = np.float32([[0, 0], 
                            [int(np.random.uniform(0.75, 0.85) * width), 0], 
                            [0, int(np.random.uniform(0.85, 0.95) * height)], 
                            [int(np.random.uniform(0.75, 0.85) * width), 
                             int(np.random.uniform(0.65, 0.75) * height)]])
    imgPts = np.float32([
    [np.random.uniform(0.10, 0.20) * width, np.random.uniform(0.20, 0.30) * height],  # 15% of width, 25% of height
    [np.random.uniform(0.75, 0.85) * width, np.random.uniform(0.15, 0.25) * height],  # 80% of width, 20% of height
    [np.random.uniform(0.05, 0.15) * width, np.random.uniform(0.65, 0.75) * height],  # 10% of width, 70% of height
    [np.random.uniform(0.85, 0.95) * width, np.random.uniform(0.65, 0.75) * height]   # 90% of width, 70% of height
    ])
    
    
    #imgPts = np.float32([[114, 151], [605, 89], [72, 420], [637, 420]])
    # need to add varibility
    #objPoints = np.float32([[0, 0], [420, 0], [0, 637], [420, 637]])
    matrix = cv2.getPerspectiveTransform(imgPts, objPoints)
    result = cv2.warpPerspective(input_image, matrix, (width, height))
    
    cv2.imwrite(r'{}/persp_{}'.format(folder,dataframe[name]), result)

    
    return pd.DataFrame({"image_path": ['{}/persp_{}'.format(folder,dataframe[name])],
                      'label': [dataframe[label]],
                      'total_path': ['{}/persp_{}'.format(folder,dataframe[name])],
                      'short_name': ['persp_{}'.format(dataframe[name])]
                      })

loc = "custom_aug" 
for index, row in train.iterrows():
    persp_train_dataframe = pd.concat([persp_train_dataframe, 
           custom_getPerspectiveTransform(row, 'label', 'short_name', 'total_path', loc, persp_train_dataframe)], sort=False)
    gs_train_dataframe = pd.concat([gs_train_dataframe, 
           custom_grey_scale(row, 'label', 'short_name', 'total_path', loc, gs_train_dataframe)], sort=False)
    #blur_train_dataframe = pd.concat([blur_train_dataframe, 
           #custom_blur(row, 'label', 'short_name', 'total_path', loc, blur_train_dataframe,blur_number)], sort=False)
    #print(row)
train = pd.concat([train, persp_train_dataframe], sort=False)
train = pd.concat([train, gs_train_dataframe], sort=False)
#train = pd.concat([train, blur_train_dataframe], sort=False)

for index, row in valid.iterrows():
    persp_valid_dataframe = pd.concat([persp_valid_dataframe, 
               custom_getPerspectiveTransform(row, 'label', 'short_name', 'total_path', loc, persp_valid_dataframe)], sort=False)
    gs_valid_dataframe = pd.concat([gs_valid_dataframe, 
           custom_grey_scale(row, 'label', 'short_name', 'total_path', loc, gs_valid_dataframe)], sort=False)
    #blur_valid_dataframe = pd.concat([blur_valid_dataframe, 
           #custom_blur(row, 'label', 'short_name', 'total_path', loc, blur_valid_dataframe,blur_number)], sort=False)
    #print(row)
valid = pd.concat([valid, persp_valid_dataframe], sort=False)
valid = pd.concat([valid, gs_valid_dataframe], sort=False)
#valid = pd.concat([valid, blur_valid_dataframe], sort=False)



In [6]:
valid

,image_path,label,total_path,short_name
1045,/train_images/1046.jpg,35,feather-in-focus/train_images/train_images/104...,1046.jpg
3856,/train_images/3857.jpg,189,feather-in-focus/train_images/train_images/385...,3857.jpg
376,/train_images/377.jpg,13,feather-in-focus/train_images/train_images/377...,377.jpg
1789,/train_images/1790.jpg,62,feather-in-focus/train_images/train_images/179...,1790.jpg
1900,/train_images/1901.jpg,66,feather-in-focus/train_images/train_images/190...,1901.jpg
...,...,...,...,...
0,custom_aug/gs_2313.jpg,83,custom_aug/gs_2313.jpg,gs_2313.jpg
0,custom_aug/gs_3845.jpg,187,custom_aug/gs_3845.jpg,gs_3845.jpg
0,custom_aug/gs_2972.jpg,116,custom_aug/gs_2972.jpg,gs_2972.jpg
0,custom_aug/gs_296.jpg,11,custom_aug/gs_296.jpg,gs_296.jpg


In [7]:
# this creates the type of augmentation that is done! 
datagen = ImageDataGenerator(
    #rotation_range=15,
    rotation_range=50,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 15,
    horizontal_flip = True,
    zoom_range = 0.20)
#look at aug from group chat 

#okay this method seems WAY better. only ever use aug images
train_generator=datagen.flow_from_dataframe(
dataframe=train,
x_col='total_path',
y_col='label',
#classes = train['label'].tolist(),
color_mode = 'rgb',
target_size = image_shape)

valid_generator=datagen.flow_from_dataframe(
dataframe=valid,
x_col='total_path',
y_col='label',
#classes = valid['label'].tolist(),
color_mode = 'rgb',
target_size = image_shape)



Found 9420 validated image filenames belonging to 200 classes.
Found 2358 validated image filenames belonging to 200 classes.


In [8]:

small_data = pd.DataFrame({'real_label': np.unique(train_image['label']), 
                           'keras_label':pd.DataFrame(np.unique(train_image['label'])).index})
class_weights = train_image.merge(small_data, left_on = 'label', right_on = 'real_label')
class_number = class_weights['keras_label']


In [ ]:
# class weights
from sklearn.utils import compute_class_weight
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(class_number),
                                        y = class_number                                                    
                                    )

class_weights = dict(zip(np.unique(class_number), class_weights))

In [ ]:
import tensorflow as tf
# attempt 100000 to get multi api to work
# Create the base model (ResNet50)
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=image_shape_full,
    pooling='avg',
    weights='imagenet'
)

# Freeze the ResNet50 model layers
base_model.trainable = False
# Define the input layer
inputs = tf.keras.Input(shape=image_shape_full)
# Pass the input through the base model
x = base_model(inputs, training=False)
# Hidden layers with BatchNormalization and Dropout
x = tf.keras.layers.Dense(Neurons, activation="relu", 
                          kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
#x = tf.keras.layers.Dense(Neurons, activation="relu")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(drop_out_rate)(x)

#x = tf.keras.layers.Dense(Neurons, activation="relu")(x)
#x = tf.keras.layers.BatchNormalization()(x)

#x = tf.keras.layers.Dense(Neurons, activation="relu")(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.Dropout(drop_out_rate)(x)

# Create the base model (ResNet50)
base_model2 = tf.keras.applications.InceptionV3(
    include_top=False,
    input_shape=image_shape_full,
    pooling='avg',
    weights='imagenet'
)

# Freeze the ResNet50 model layers
base_model2.trainable = False
# Pass the input through the base model
x2 = base_model2(inputs, training=False)
# Hidden layers with BatchNormalization and Dropout
x2 = tf.keras.layers.Dense(Neurons, activation="relu", 
                          kernel_regularizer=tf.keras.regularizers.l2(0.01))(x2)
#x2 = tf.keras.layers.Dense(Neurons, activation="relu")(x2)
x2 = tf.keras.layers.BatchNormalization()(x2)
x2 = tf.keras.layers.Dropout(drop_out_rate)(x2)


#x2 = tf.keras.layers.Dense(Neurons, activation="relu")(x2)
#x2 = tf.keras.layers.BatchNormalization()(x2)

#x2 = tf.keras.layers.Dense(Neurons, activation="relu")(x2)
#x2 = tf.keras.layers.BatchNormalization()(x2)
#x2 = tf.keras.layers.Dropout(drop_out_rate)(x2)


# combine 
weighted_average = tf.keras.layers.Average()([x,x2])
#combined_predictions = tf.keras.layers.Concatenate()([dense2_res, dense2_invepV3])

# Output layer
outputs = tf.keras.layers.Dense(output_Neurons, activation="softmax", name="output_layer")(weighted_average)

# Create the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Adam optimizer
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)

# Compile the model 
model.compile(
    loss="categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=["accuracy"]
)

model.summary()

In [ ]:

# this is supposed to help with over-fitting
#Setting the early_stop to avoid overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=patience1,
    min_delta=0.001,
    restore_best_weights=True,)

history = model.fit(
    train_generator,
    epochs= number_epcho1,
   # steps_per_epoch=len(train_generator),
    validation_data=valid_generator,
    #validation_steps=int(0.2 * len(valid_generator)),
    callbacks=[early_stop],
    class_weight=class_weights
)


In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:

x = reversed(range(always_keep_percent_locked, start_unlocking_at + decay_by ,decay_by))
for n in x:
    continue
    base_model.trainable = True
    #number of base model layers 80 % of base model 
    fine_tune_at = (len(base_model.layers) // 100) * n
    # 176 layers in base 1
    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    base_model2.trainable = True    
    #number of base model layers 80 % of base model 
    fine_tune_at = (len(base_model2.layers) // 100) * n
    # 312 layers in base_model2
    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model2.layers[:fine_tune_at]:
        layer.trainable = False  
    # Now that we will allow some layers to be unfreezed, it's better to decrease the learning rate to avoid dramatic changes in those 
    adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate/100)

    # Compile the model again
    model.compile(
        #loss = "sparse_categorical_crossentropy",
        loss="categorical_crossentropy",
        optimizer=adam_optimizer,
        metrics=["accuracy"]
    )

    early_stop = tf.keras.callbacks.EarlyStopping(
        patience=patience2,
        min_delta=0.0001,
        restore_best_weights=True,)

    total_epochs = number_epcho1 + number_epcho1

    history_fine = model.fit(
                            train_generator,
                             epochs=total_epochs,
                             initial_epoch=history.epoch[-1],
                             steps_per_epoch=len(train_generator),
                             validation_data=valid_generator,
                             validation_steps=int(0.25 * len(valid_generator)),
                            callbacks=[early_stop],
        class_weight=class_weights
    )



In [ ]:
n = 80
base_model.trainable = True
#number of base model layers 80 % of base model 
fine_tune_at = (len(base_model.layers) // 100) * n
# 176 layers in base 1
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

base_model2.trainable = True    
#number of base model layers 80 % of base model 
fine_tune_at = (len(base_model2.layers) // 100) * n
# 312 layers in base_model2
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model2.layers[:fine_tune_at]:
    layer.trainable = False  
# Now that we will allow some layers to be unfreezed, it's better to decrease the learning rate to avoid dramatic changes in those 
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate/100)

# Compile the model again
model.compile(
    #loss = "sparse_categorical_crossentropy",
    loss="categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=["accuracy"]
)

early_stop = tf.keras.callbacks.EarlyStopping(
    patience=patience2,
    min_delta=0.0001,
    restore_best_weights=True,)

total_epochs = number_epcho1 + number_epcho1

history_fine = model.fit(
                        train_generator,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                        # steps_per_epoch=len(train_generator),
                         validation_data=valid_generator,
                         #validation_steps=int(0.25 * len(valid_generator)),
                        callbacks=[early_stop],
    class_weight=class_weights
)


In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
current_directory = "feather-in-focus"
model.save('{}/keras_final.keras'.format(current_directory))

# image folder
folder_path = '{}/test_images/test_images/'.format(current_directory)
# path to model
model_path = '{}/keras_final.keras'.format(current_directory)
# dimensions of images
img_width, img_height = 224, 224

In [ ]:
from keras.models import load_model
import keras.utils as image
# load the trained model
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
model_path = '{}/keras_final.keras'.format(current_directory)
model = load_model(model_path)
model.compile(loss="categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=["accuracy"])

In [ ]:
name = list()
images = []
folder_path = 'feather-in-focus/test_images/test_images'  # Your folder path containing images
for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)
    # Check if the item is a file (not a directory)
    if os.path.isfile(img_path):
        name.append(img_name)
        img = image.load_img(img_path, target_size=image_shape)
        img = image.img_to_array(img)
        
        
        #img_array = image.img_to_array(img)
        #img = img.astype('float32') / 255.0  # Normalize pixel values

# Reshape the image to a 4D tensor with a single sample (to match model input shape)
        #img = img.reshape((1,) + img.shape)
        
        
        img = np.expand_dims(img, axis=0)
        images.append(img)
        # Process the image further as needed
    else:
        # Skip directories
        continue
        



In [ ]:
# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict(images, batch_size=10)
classes_x=np.argmax(classes,axis=1)

In [ ]:
final_df = pd.DataFrame(
    {'id': name,
     'label_predict': classes_x
    })

final_df['id2'] = "/test_images/"+final_df['id']
final_df['label_predict'] = final_df['label_predict'].astype(int)
final_df = final_df[['label_predict','id2']]

idea = np.sort(train_image['label'].unique())
idea2 = pd.DataFrame(idea)
idea2['index1'] = idea2.index
idea2.columns = ['values', 'index1']
idea2['values'] = idea2['values'].astype(int)

final_df4 = final_df.merge(test_image_path[['image_path','id']], left_on = 'id2', right_on='image_path')

final_df4 = final_df4[['id','label_predict']]

final_df2 = final_df4.merge(idea2, how = 'left', left_on = 'label_predict', right_on = 'index1')
final_df3 = final_df2[['id','values']]
#two issues the class values arent right
final_df3.columns = ['id','label']
final_df3


In [ ]:
final_df3.to_csv('output_folder/output.csv', index=False)

In [ ]:
import json

# Define the metadata for your dataset
metadata = {
    "title": "output.csv",
    "id": "uvajonathon",
    "licenses": [
        {
            "name": "License Name",
            "id": "uvajonathon",
            "url": "URL to License if applicable"
        }
    ],
    "description": "Description of your dataset",
    "keywords": ["keyword1", "keyword2"],
    "language": "en",
    "isPrivate": False,  # Set to True or False based on privacy
    "collaborators": ["uvajonathon"],
    "data": [
        {
            "description": "Description of the data",
            "name": "output.csv"
        }
    ]
}

# Save the metadata to dataset-metadata.json
file_path = 'output_folder/dataset-metadata.json'  # Specify the path where you want to save the file

with open(file_path, 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

print(f"Dataset metadata saved to {file_path}")


In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
# Set the path to your Kaggle API credentials file (kaggle.json)
os.environ['KAGGLE_CONFIG_DIR'] = '/gpfs/home2/scur2079/'
# Instantiate the Kaggle API
api = KaggleApi()
# Replace 'competition-name' with the competition name (as shown in the competition URL)
competition_name = 'feather-in-focus'
# Replace 'file-path.csv' with the path to your CSV file
file_path = '/gpfs/home2/scur2079/output_folder'

In [ ]:

# Specify the competition to which you want to upload the file
api.competition_name = competition_name
# Upload the CSV file to the competition
api.dataset_create_new(file_path)

In [10]:
model

In [15]:
# Train your Keras model as described in your provided code

# Assuming you've trained your Keras model and stored it in a variable 'model'
from keras.models import Model
# Extract features using the Keras Functional API
# Create a new model that takes the same input as the previous model but outputs the output of the desired layer
layer_name='output-layer'
feature_extraction_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)


# 
# Get features for training and validation data
train_features = feature_extraction_model.predict(train_generator)  # Replace train_data with your actual training data
#valid_features = feature_extraction_model.predict(valid)  # Replace valid_data with your actual validation data


2023-12-05 15:14:52.610143: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


  1/197 [..............................] - ETA: 17:14

2023-12-05 15:14:54.857331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


197/197 [==============================] - 70s 331ms/step


In [25]:
type(train_generator)

keras.preprocessing.image.DataFrameIterator

In [29]:
def extract_labels(image, label):
    return label

# Iterate through the generator to extract labels
labels = []
for images, labels_batch in train_generator:
    labels.extend(extract_labels(images, labels_batch))

TypeError: extract_labels() missing 1 required positional argument: 'label'

In [32]:
labels=list()

In [19]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier()  # You can set hyperparameters as needed
xgb_model.fit(train_features,labels)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0], got [dict_keys(['1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 12.3 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [24]:
name = list()
images = []
from keras.models import load_model
import keras.utils as image
folder_path = 'feather-in-focus/test_images/test_images'  # Your folder path containing images
for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)
    # Check if the item is a file (not a directory)
    if os.path.isfile(img_path):
        name.append(img_name)
        img = image.load_img(img_path, target_size=image_shape)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        images.append(img)
        # Process the image further as needed
    else:
        # Skip directories
        continue
        
# stack up images list to pass for prediction
images = np.vstack(images)
classes = feature_extraction_model.predict(images, batch_size=10)
classes_x=np.argmax(classes,axis=1)

NameError: name 'name' is not defined

In [23]:


valid_features[0]

200

In [21]:
classes_x=np.argmax(valid_features,axis=1)
final_df = pd.DataFrame(
    {'id': name,
     'label_predict': classes_x
    })

final_df['id2'] = "/test_images/"+final_df['id']
final_df['label_predict'] = final_df['label_predict'].astype(int)
final_df = final_df[['label_predict','id2']]

idea = np.sort(train_image['label'].unique())
idea2 = pd.DataFrame(idea)
idea2['index1'] = idea2.index
idea2.columns = ['values', 'index1']
idea2['values'] = idea2['values'].astype(int)

final_df4 = final_df.merge(test_image_path[['image_path','id']], left_on = 'id2', right_on='image_path')

final_df4 = final_df4[['id','label_predict']]

final_df2 = final_df4.merge(idea2, how = 'left', left_on = 'label_predict', right_on = 'index1')
final_df3 = final_df2[['id','values']]
#two issues the class values arent right
final_df3.columns = ['id','label']
final_df3

NameError: name 'name' is not defined